### Entranamiento / Evaluación

#### 💪 Training

In [1]:
!pip install --upgrade pip
!pip uninstall numpy scipy scikit-learn -y
!pip install numpy scipy pandas scikit-learn
!pip install --upgrade pip


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   -------- ------------------------------- 0.4/1.8 MB 8.1 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.8 MB 8.8 MB/s eta 0:00:01
   ------------------------------ --------- 1.4/1.8 MB 10.8 MB/s eta 0:00:01
   ---------------------------------- ----- 1.6/1.8 MB 9.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
Found existing installation: scipy 1.13.0
Uninstalling scipy-1.13.0:
  Successfully uninstalled scipy-1.13.0
Found existing installation: scikit-learn 1.5.0
Uninstalling scikit-learn-1.5.0:
  Successfully uninstalled scikit-learn-1.5.0
   ---------------------------------------- 0.0/16.6 MB ? eta 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.59.1 requires numpy<1.27,>=1.22, but you have numpy 2.0.1 which is incompatible.
rustworkx 0.14.2 requires numpy<2,>=1.16.0, but you have numpy 2.0.1 which is incompatible.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.0.1 which is incompatible.
ydata-profiling 4.8.3 requires numpy<2,>=1.16.0, but you have numpy 2.0.1 which is incompatible.
ydata-profiling 4.8.3 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.0 which is incompatible.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
proc_data_path='../data/processed/road_traffic_accidents_dataset_proc.csv'
train_data = pd.read_csv(proc_data_path)


In [4]:
X = train_data.drop('accident_severity', axis=1)
y = train_data['accident_severity']




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

from sklearn.metrics import accuracy_score

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X, y, cv=5, scoring='accuracy')

print('Cross-validation scores:{}'.format(scores))

print('Average cross-validation score: {}'.format(scores.mean()))

import joblib

# Save the model as a pickle file
filename = '../models/rf_model.pkl'
joblib.dump(rf, filename)

# Load the model from the file
rf_model = joblib.load(filename)

# Use the loaded model to make predictions
y_pred = rf_model.predict(X_test)

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

print('Classification report : ')

print(classification_report(y_test, y_pred))

print('Cross-validation scores :')

scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')

print(scores)

print('Average cross-validation score : {}'.format(scores.mean()))



Model accuracy score with 10 decision-trees : 0.8754
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       1.00      0.04      0.09        67
           2       0.87      1.00      0.93       503

    accuracy                           0.88       578
   macro avg       0.62      0.35      0.34       578
weighted avg       0.88      0.88      0.82       578



C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classi

Cross-validation scores:[0.85640138 0.85294118 0.85294118 0.85294118 0.85961872]
Average cross-validation score: 0.8549687261998284
Model accuracy score with 10 decision-trees : 0.8754
Classification report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       1.00      0.04      0.09        67
           2       0.87      1.00      0.93       503

    accuracy                           0.88       578
   macro avg       0.62      0.35      0.34       578
weighted avg       0.88      0.88      0.82       578

Cross-validation scores :


C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classi

[0.85640138 0.85294118 0.85294118 0.85294118 0.85961872]
Average cross-validation score : 0.8549687261998284


#### 🎯 Predict